In [1]:
# ==============================================================
# 06 — LATENCY ANALYSIS
# Electrum Observatory
# ==============================================================

import json
import pandas as pd
import plotly.express as px
from pathlib import Path

DATA_DIR = Path("../../data")

# ------------------------------
# Load online peers (contains latency)
# ------------------------------

with open(DATA_DIR / "online_peers" / "online_peers.json", "r") as f:
    online_raw = json.load(f)

# Convert to dataframe
df = pd.DataFrame(online_raw)

print("Rows loaded:", len(df))
df.head()

Rows loaded: 1665


,host,port,protocol,latency_ms,version_raw,banner_raw
0,137.184.244.174,50001,tcp,5856.29,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."
1,194.233.69.180,50001,tcp,6076.51,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."
2,35.189.13.187,50001,tcp,7056.47,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.18.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."
3,23.155.96.131,50002,ssl,1198.84,"{""id"":1,""jsonrpc"":""2.0"",""result"":[""electrs-esp...","{""id"":2,""jsonrpc"":""2.0"",""result"":""\""Welcome to..."
4,159.69.244.130,50002,ssl,1031.90,"{""jsonrpc"":""2.0"",""result"":[""ElectrumX 1.16.0"",...","{""jsonrpc"":""2.0"",""result"":""You are connected t..."


In [2]:
# Ensure numeric latency
df["latency_ms"] = pd.to_numeric(df["latency_ms"], errors="coerce")

# Remove entries with missing latency
df = df.dropna(subset=["latency_ms"])

print("Valid latency rows:", len(df))
df["latency_ms"].describe()

Valid latency rows: 1665


count    1665.000000
mean     1939.517730
std      1214.144065
min       231.990000
25%      1109.340000
50%      1779.400000
75%      2287.580000
max      7929.060000
Name: latency_ms, dtype: float64

In [3]:
fig = px.histogram(
    df,
    x="latency_ms",
    nbins=50,
    title="Latency Distribution (ms)",
    template="plotly_dark",
    opacity=0.85,
)

fig.update_layout(
    xaxis_title="Latency (ms)",
    yaxis_title="Count",
    height=450,
)

fig.show()

In [4]:
fig_density = px.density_contour(
    df,
    x="latency_ms",
    title="Latency Density Curve",
    template="plotly_dark",
)

fig_density.update_traces(contours_coloring="fill", opacity=0.6)
fig_density.update_layout(height=450, xaxis_title="Latency (ms)")

fig_density.show()

In [5]:
EXPORT_DIR = Path("../../site/public/results")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

fig_density.write_image(str(EXPORT_DIR / "latency_density.png"), scale=2)
print("Exported to site/public/results/latency_density.png")

Exported to site/public/results/latency_density.png


In [6]:
fig = px.box(
    df,
    y="latency_ms",
    title="Latency Outlier Detection",
    template="plotly_dark",
)

fig.update_layout(height=350)
fig.show()

In [7]:
if "country" in df.columns:
    fig = px.box(
        df,
        x="country",
        y="latency_ms",
        points="all",
        title="Latency by Country",
        template="plotly_dark",
    )

    fig.update_layout(height=650, xaxis_tickangle=45)
    fig.show()
else:
    print("No 'country' column found — skip country-based analysis.")

No 'country' column found — skip country-based analysis.


In [8]:
if "lat" in df.columns and "lon" in df.columns:
    fig = px.scatter_geo(
        df,
        lat="lat",
        lon="lon",
        color="latency_ms",
        hover_name="host",
        title="Global Latency Heatmap",
        template="plotly_dark",
        color_continuous_scale="Turbo",
    )
    
    fig.update_layout(height=500)
    fig.show()
else:
    print("No coordinates available — skipping map scatter.")

No coordinates available — skipping map scatter.


In [9]:
import numpy as np
from sklearn.cluster import KMeans

X = df[["latency_ms"]].dropna()

kmeans = KMeans(n_clusters=3, n_init="auto").fit(X)
df["latency_cluster"] = kmeans.labels_

fig = px.histogram(
    df,
    x="latency_ms",
    color="latency_cluster",
    nbins=50,
    title="Latency Clusters (KMeans)",
    template="plotly_dark",
)

fig.show()